In [ ]:
import re
from dataclasses import dataclass
import itertools
from typing import TypeAlias

Point: TypeAlias = tuple[int, int]

@dataclass
class Beacon:
    pos: Point

@dataclass
class Sensor:
    pos: Point
    beacon: Beacon
    distance_to_beacon: int

def manhattan_distance(p1: Point, p2: Point):
    return abs(p1[0] - p2[0]) + abs(p1[1] - p2[1])

def parse():
    sensors = []
    for line in open("input.txt"):
        m = re.match("Sensor at x=(?P<sx>-?\d+), y=(?P<sy>-?\d+): closest beacon is at x=(?P<bx>-?\d+), y=(?P<by>-?\d+)", line)
        p1 = (int(m.group("sx")), int(m.group("sy")))
        p2 = (int(m.group("bx")), int(m.group("by")))
        md = manhattan_distance(p1, p2)
        sensors.append(Sensor(p1, Beacon(p2), md))

    return sensors

def inside(s: Sensor, pos: Point) -> bool:
    return s.distance_to_beacon >= manhattan_distance(s.pos, pos)

def is_in_range(x, r):
    return r[0] <= x and x <= r[1]

def ranges_touch(r1, r2):
    return is_in_range(r1[0], r2) or is_in_range(r1[1], r2)

def merge_ranges(r1, r2):
    if ranges_touch(r1, r2):
        return (min(r1[0], r2[0]), max(r1[1], r2[1]))
    return None

def considers_row(s: Sensor, row: int) -> bool:
    return is_in_range(row, (s.pos[1] - s.distance_to_beacon, s.pos[1] + s.distance_to_beacon))

def pretty_print(sensors: list[Sensor]):
    minx, maxx = min([s.pos[0] - s.distance_to_beacon for s in sensors]), max([s.pos[0] + s.distance_to_beacon for s in sensors])
    miny, maxy = min([s.pos[1] - s.distance_to_beacon for s in sensors]), max([s.pos[1] + s.distance_to_beacon for s in sensors])
    
    sensorposs = set([sensor.pos for sensor in sensors])
    beaconspos = set([sensor.beacon.pos for sensor in sensors])

    print("   ", end="")
    for x in range(minx, maxx + 1):
        print(f"{x:^3}", end= "")
    print()

    for y in range(miny, maxy + 1):
        print(f"{y:>3}", end="")
        for x in range(minx, maxx + 1):
            ss = [s for s in sensors if inside(s, (x,y))]
            print(f'{"." if not ss else "S" if (x,y) in sensorposs else "B" if (x,y) in beaconspos else "#":^3}', end="")
        print()

    print(" " * maxx * 3, (maxx, maxy))  

def count_convered_spots(sensors: list[Sensor], row):
    ranges = set()
    for s in sensors:
        if not considers_row(s, row):
            continue
        x_distance_to_edge = (s.distance_to_beacon - manhattan_distance((s.pos[0], row), s.pos))
        ranges = ranges.add((s.pos[0] - x_distance_to_edge, s.pos[0] + x_distance_to_edge))

    stop = False
    while not stop: 
        stop = True
        for (r1, r2) in list([(r1, r2) for (r1, r2) in itertools.product(ranges.copy(), ranges.copy()) if ranges_touch(r1, r2) and r1 != r2]):
            if r1 in ranges: ranges.remove(r1)
            if r2 in ranges: ranges.remove(r2)
            ranges.add(merge_ranges(r1, r2))
            stop = False

    return ranges

In [ ]:
sensors = parse()
#pretty_print(sensors)
row = 2000000
ranges = count_convered_spots(sensors, row)
sum([xmax - xmin + 1 for xmin, xmax in ranges]) - len(set([s.beacon.pos for s in sensors if s.beacon.pos[1] == row]))


In [ ]:
sensors = parse()
# Ugh 30+ min
for y in range(20):
    r = count_convered_spots(sensors, y)
    if len(r) > 1:
        magic_row = y
        break

In [ ]:
sensors = parse()
magic_x = sorted(count_convered_spots(sensors, magic_row), key=lambda x: x[1])[0][1] + 1
magic_x * 4000000 + magic_row